In [1]:
import pandas as pd

In [20]:
species = {21725:'Boreogadus saida',23041:'Mallotus catervarius', 21110:'Clupea pallasii',21735:'Eleginus gracilis', 21744:'Gadus chalcogrammus', 21720:'Gadus macrocephalus'}
cur = pd.read_csv('../mbaCatchResults_2017.csv')
cur = cur[['INTERVAL','END_LATITUDE','END_LONGITUDE','MEAN_BOTTOM_DEPTH','LAYER','SPECIES_CODE','LENGTH','NUMBERS','NUMBERS_NM2']]
cur = cur.rename(columns={'INTERVAL':'Interval','END_LATITUDE':'Latitude',
                    'END_LONGITUDE':'Longitude','MEAN_BOTTOM_DEPTH':'Mean Bottom Depth',
                    'LAYER':'Depth Layer','SPECIES_CODE':'Species','LENGTH':'Length Bin',
                    'NUMBERS':'Numbers', 'NUMBERS_NM2':'Numbers nmi^-2'})
cur = cur.replace({'Species':species})
cur = cur[(cur.Longitude!=999) | (cur.Latitude!=999)]
cur.to_csv('Arctic_EIS_Acoustic_trawl_complete_2017.csv')

In [21]:
species = {21725:'Boreogadus saida',23041:'Mallotus catervarius', 21110:'Clupea pallasii',21735:'Eleginus gracilis', 21744:'Gadus chalcogrammus', 21720:'Gadus macrocephalus'}
cur = pd.read_csv('../mbaCatchResults_2019.csv')
cur = cur[['INTERVAL','END_LATITUDE','END_LONGITUDE','MEAN_BOTTOM_DEPTH','LAYER','SPECIES_CODE','LENGTH','NUMBERS','NUMBERS_NM2']]
cur = cur.rename(columns={'INTERVAL':'Interval','END_LATITUDE':'Latitude',
                    'END_LONGITUDE':'Longitude','MEAN_BOTTOM_DEPTH':'Mean Bottom Depth',
                    'LAYER':'Depth Layer','SPECIES_CODE':'Species','LENGTH':'Length Bin',
                    'NUMBERS':'Numbers', 'NUMBERS_NM2':'Numbers nmi^-2'})
cur = cur.replace({'Species':species})
cur = cur[(cur.Longitude!=999) | (cur.Latitude!=999)]
cur.to_csv('Arctic_EIS_Acoustic_trawl_complete_2019.csv')

# For Kuletz

In [32]:
from tqdm.notebook import tqdm
import pandas as pd

df = pd.read_csv('../mbaCatchResults_2019.csv')
a = df.groupby(['INTERVAL','END_LATITUDE','END_LONGITUDE']).mean().reset_index()[['INTERVAL','END_LATITUDE','END_LONGITUDE']]
b = a[(a.END_LATITUDE<999)&(a.END_LONGITUDE<999)]

intervals = sorted(b.INTERVAL.unique())
specs = df.SPECIES_CODE.unique()

sNames = {21725:'Boreogadus saida',23041:'Mallotus catervarius', 21110:'Clupea pallasii',21735:'Eleginus gracilis', 21744:'Gadus chalcogrammus', 21720:'Gadus macrocephalus'}
sgLab = {0:'0-5.5 cm',5:'5.5-10.5 cm', 10:'>10.5 cm'}
dzLab = {0:'6.5-20m',4:'20-40m', 8:'40-60m',12:'60-80m',16:'>80m'}

minD = [0,4,8,12,16]
maxD = [4,8,12,16,100]
minL = [0,5,10]
maxL = [5,10,100]


num, ints, spec,dz,sg=[],[],[],[],[]

for i in tqdm(intervals):
    for s in specs:
        if s == 21740:
            continue
        for ctD in range(len(maxD)):
            for ctL in range(len(maxL)):
                if s == 21744:
                    num.append(df[(df.INTERVAL==i)&(df.SPECIES_CODE.isin([21740,21744]))&(df.LAYER>minD[ctD])&(df.LAYER<=maxD[ctD])&(df.LENGTH>minL[ctL])&(df.LENGTH<=maxL[ctL])].sum().NUMBERS_NM2)
                else:
                    num.append(df[(df.INTERVAL==i)&(df.SPECIES_CODE==s)&(df.LAYER>minD[ctD])&(df.LAYER<=maxD[ctD])&(df.LENGTH>minL[ctL])&(df.LENGTH<=maxL[ctL])].sum().NUMBERS_NM2)
                ints.append(i)
                spec.append(s)
                dz.append(minD[ctD])
                sg.append(minL[ctL])

In [41]:
sNames = {21725:'Boreogadus saida',23041:'Mallotus catervarius', 21110:'Clupea pallasii',21735:'Eleginus gracilis', 21740:'Gadus chalcogrammus',21744:'Gadus chalcogrammus', 21720:'Gadus macrocephalus'}
test = pd.DataFrame({'Species':[sNames[k] for k in spec],'Size_group':[sgLab[k] for k in sg],'Depth_zone':[dzLab[k] for k in dz],'Interval':ints,'Density_fish_nmi_squared':num})
test['Density_fish_km_squared'] = test.Density_fish_nmi_squared*(1/(1.852*1.852))
sgLab = {0:'0-5.5 cm',5:'5.5-10.5 cm', 10:'>10.5 cm'}
dzLab = {0:'6.5-20m',4:'20-40m', 8:'40-60m',12:'60-80m',16:'>80m'}
test2 = pd.merge(test, b.rename(columns={'INTERVAL':'Interval','END_LATITUDE':'Latitude','END_LONGITUDE':'Longitude'}),on='Interval')
test2[['Species', 'Size_group', 'Depth_zone', 'Interval','Latitude','Longitude','Density_fish_nmi_squared', 'Density_fish_km_squared']].to_csv('Arctic_IES_Acoustic_trawl_Kuletz_2019_v3.csv',index=False)